In [1]:
from edb import ElectrolyteDB
import os

edb = ElectrolyteDB("edb_workflow.sqlite")

1. For this example, we will choose the Hydronium form and a basic solution.

2. We add H2O and KHCO3 as known species, and indicate that H2O is the solvent

    * These are added to the Unsorted component list


In [2]:
from dataclasses import dataclass, field
from typing import Set

@dataclass
class SpeciesManager:
    """Convenience class to help keep track of different lists of species.
    """
    unsorted: Set = field(default_factory=set)
    solvent: Set = field(default_factory=set)
    apparent: Set = field(default_factory=set)
    cation: Set = field(default_factory=set)
    anion: Set = field(default_factory=set)
    liquid: Set = field(default_factory=set)

    def move(self, name, from_list, to_list):
        """Move a species from one list to another.
        """
        err = None
        src = getattr(self, from_list, None)
        if src is None:
            err = f"unknown source list '{src}'"
        else:
            dst = getattr(self, to_list, None)
            if dst is None:
                err = f"unknown destination list '{to_list}'"
            else:
                if name not in src:
                    err = f"component '{name}' not found in list '{from_list}'"
                else:
                    if name in dst:
                        err = f"component '{name}' is in source list '{from_list}' and " \
                              f"destination list '{to_list}'"
                    else:
                        src.remove(name)
                        dst.add(name)
                        print(f"Moved '{name}' from {from_list} => {to_list}")
        if err is not None:
            raise ValueError(err)

    def all(self, unsorted=True):
        s = self. solvent | self.apparent | self.cation | self.anion
        if unsorted:
            s |= self.unsorted
        return s


3. Lookup component types

    * `lookup_component(H2O).type >>> "molecular"`
        * As water is indicated as a solvent, confirm that type is "molecular" and add to Solvent list and remove from "Unsorted"
        * `lookup_component(KHCO3).type >>> "salt"`
        * Add to "Apparent" list and remove from "Unsorted"


In [3]:
sp = SpeciesManager(unsorted={"H2O", "KHCO3"})


In [4]:
# for each sub-list, keys are arbitrary names like "S001"
reactions = {'dissociation': {}}

In [5]:
comp = edb.get_component("H2O")
print(f"Type for component H2O => {comp.type}")
assert comp.type == "molecular"
sp.move("H2O", "unsorted", "solvent")
comp = edb.get_component("KHCO3")
print(f"Type for component KHCO3 => {comp.type}")
assert comp.type == "salt"
sp.move("KHCO3", "unsorted", "apparent")

Type for component H2O => molecular
Moved 'H2O' from unsorted => solvent
Type for component KHCO3 => salt
Moved 'KHCO3' from unsorted => apparent



4. Iterate over "Apparent" list and lookup dissociation species
    * `lookup_component(KHCO3).dissociation >>> [K+, OH-]`, ID for dissociation reaction (call it S001 for now)


In [6]:
for s in sp.apparent:
    comp = edb.get_component(s)
    print(f"{s} dissociation: {comp.dissociation}")


KHCO3 dissociation: ['K+', 'HCO3-']



5. We will not exclude this reaction
    * Add S001 to "Dissociation" reactions list
    * Add K+ and OH- to "Unsorted" component list


In [7]:
reactions["dissociation"]["S001"] = comp.dissociation
sp.unsorted |= set(comp.dissociation)
sp.unsorted

{'HCO3-', 'K+'}

6. Lookup new species

    * `lookup_component(K+).type >>> "cation"`
    * `lookup_component(HCO3-).type >>> "anion"`
    * Sort species and remove from "Unsorted" list


In [8]:
for formula in sp.unsorted.copy():
    comp = edb.get_component(formula)
    print(f"Type for {formula} => {comp.type}")
    # not sure what 'Sort species' means -- going to assume list-for-type, for now
    sp.move(formula, "unsorted", comp.type)

Type for K+ => cation
Moved 'K+' from unsorted => cation
Type for HCO3- => anion
Moved 'HCO3-' from unsorted => anion



7. Lookup reactions from database – should find only 1 new reaction at this point:

    * Water Self-Ionization, Hydronium form (H2O <-> H30+ + OH-)
    * Add this to "Potential" reactions list (call it L001:hydronium for now)


In [9]:
print(sp.all())
rct = edb.get_reactions({'H2O'}, form_filter={'hydronium'})
#assert len(rct) == 1
print()
for r in rct:
    print(r.as_text())
    print()

{'H2O', 'KHCO3', 'HCO3-', 'K+'}

ID         : 1
Description: Self ionization of water
Formula    : H2O <-> H3O+ + OH-
Form       : hydronium
Conditions : occurs naturally in all aqueous systems
Acid/Base  : any
pKa at 298K: 13.995



8. No custom reactions to add
9. We will not exclude the water self-ionization reaction

    * Add water self-ionisation to "Liquid" reactions list


In [10]:
sp.liquid.add(rct[0])

10. Lookup participating species for new reactions

    * `lookup_reaction(L001:hydronium).participating >>> H2O, H3O+, OH-`
    * H3O+ and OH- are new species so add to "Unsorted" component list
    * Remove L001:hydronium from "Potential" reaction list and add to "Liquid" list – need to do a `lookup_reaction(L001).type >>> "Liquid"`

In [16]:
l001 = rct[0]
reactions = edb.get_reactions(l001.components, form_filter={"hydronium"})
for r in reactions:
    print(r)
    for c in r.components:
        print(c)

ID         : 1
Description: Self ionization of water
Formula    : H2O <-> H3O+ + OH-
Form       : hydronium
Conditions : occurs naturally in all aqueous systems
Acid/Base  : any
pKa at 298K: 13.995
(H2O)2
H3O+
OH-





11. Lookup type of new species (I.e those in "Unsorted")

    * `lookup_component(H3O+).type >>> "cation"`
    * `lookup_component(OH-).type >>> "anion"`
    * Move components to appropriate lists

12. (Repeat Step 7) – Lookup reactions from database – should find the following new reactions at this point:

    * L002:hydroxide: HCO3- ↔ CO2 + OH-
    * L003:hydroxide: HCO3- + OH-↔ CO32- + H2O
    * S002: KOH ↔ K+ + OH-

13. (Repeat Step 8) – No new custom reactions
14. (Repeat Step 9) – Do not exclude any of these reactions
15. (Repeat Step 10) – Look up new species

    * `lookup_reaction(L002:hydroxide).participating >>> HCO3-, CO2, OH-`
    * `lookup_reaction(L003:hydroxide).participating >>> HCO3-, OH-, CO3--, H2O`
    * `lookup_reaction(S002).participating >>> KOH, K+, OH-`
    * New species are CO2, CO32-, and KOH; add to "Unsorted"
    * Add  L002:hydroxide and L003:hydroxide to "Liquid" list and S002 to "Dissociation" list

16. (Repeat Step 11) - Lookup type of new species (I.e those in "Unsorted")

    * `lookup_component(CO2).type >>> "molecular"`
    * `lookup_component(CO3--).type >>> "anion"`
    * `lookup_component(KOH).type >>> "salt"`
    * Move components to appropriate lists

17. (Repeat Step 7 x2) – Lookup reactions from database – should find the following new reactions at this point:•

    * L004:hydronium: H2CO3 +H2O↔ HCO3- + H3O+
    * S003: K2CO3 ↔ 2K+ + CO32-

18. (Repeat Step 8 x2) – No new custom reactions
19. (Repeat Step 9 x2) – We will exclude L004:hydronium as it is only significant at low pH

    * Move L004:hydronium to "Excluded" list
    * Based on expected pH range from Step 1, tool should help identify this
    * `lookup_reaction(L004).conditions >>> some information on when this reaction is important`,
       Should be able to identify that a basic solution can probably ignore this.
    * Possible lookup: `lookup_reaction(L004).pKa >>> 3.6`

20. (Repeat Step 10 x2) – Look up new species

    * `lookup_reaction(S003).participating >>> K2CO3, K+, CO3--`
    * One new species (K2CO3), add to unsorted list•
    * Add S002 to "Dissociation" list

21. (Repeat Step 11 x2) - Lookup type of new species (I.e those in "Unsorted")

    * `lookup_component(K2CO3).type >>> "salt"`
    * Move K2CO3 to "Apparent" list

22. (Repeat Step 7 x3) – Lookup reactions from database – should find no new reactions at this point,
    so we can finish iterating

23. (Step 13) – Verify reaction network

    * Lookup stoichiometry for all reactions and build a stoichiometric matrix
    * `lookup_reaction(S001).stoichiometry >>> {KHCO3: -1, K+: +1, HCO3-: +1}, etc.`
    * Check that every ion in the "Anion" and "Cation" has a reaction in "Apparent" reaction list that it takes part in.

24. (Step 14) – Start writing definition dict. Additional data that will be needed here:

    * Charge for ionic species (and some other types); e.g. `lookup_component(K+).charge >>> +1`
    * Molecular weight: `lookup_component(K+).molecular_weight >>> 39.0983`
    * Probably more, but for now these should be simple look ups of single values
       (we won’t deal with parameter sets for now)